# Платформа Airbnb в Берлине

Таблица `listings` содержит полную информацию о каждом объявлении Airbnb, включая описания, характеристики и средние оценки по отзывам. Таблица содержит много столбцов, только нужные перечислены в задачах.

Таблица `calendar_summary` предоставляет информацию о доступности и ценах на жильё по дням.

- **listing_id**: Идентификатор объявления
- **date**: Дата
- **available**: Доступность жилья в указанный день (t/f)
- **price**: Цена за ночь

Таблица `reviews` включает отзывы по объявлениям.

- **listing_id**: Идентификатор объявления
- **id**: Идентификатор отзыва
- **date**: Дата отзыва
- **reviewer_id**: Идентификатор автора отзыва
- **reviewer_name**: Имя автора отзыва
- **comments**: Текст отзыва

### 1. Какой месяц и год регистрации новых хостов на Airbnb оказался самым популярным?
- host_id – идентификатор хозяина
- host_since – дата регистрации как хост

```sql
SELECT toStartOfMonth(toDateOrNull(host_since)) AS max_date
FROM listings
GROUP BY max_date
ORDER BY COUNT(DISTINCT host_id) DESC
LIMIT 1
```

### 2. Какая средняя частота ответа среди обычных хостов и суперхостов на Airbnb?
- host_response_rate – частота ответа
- host_is_superhost – является ли суперхозяином


```sql
SELECT host_is_superhost, 
       AVG(host_response_rate) AS AvgResp
FROM
    (SELECT DISTINCT host_id,
                     host_is_superhost,
                     toInt32OrNull(replaceAll(host_response_rate, '%', '')) AS host_response_rate
    FROM listings) as sub
WHERE host_is_superhost != '' 
GROUP BY host_is_superhost 
```

### 3. Какую среднюю цену за ночь устанавливает каждый хост на Airbnb? Идентификаторы сдаваемого жилья объедините в отдельный массив. 
- host_id – идентификатор хозяина
- id – идентификатор жилья
- price – цена за ночь в конкретном месте


```sql
SELECT host_id,
       AVG(toFloat32OrNull(replaceRegexpAll(price, '[$,]', ''))) AS avg_price,
       groupArray(id) as ids
FROM listings
GROUP BY host_id
ORDER BY avg_price DESC, host_id DESC
```

### 4. Расчитайте разницу между максимальной и минимальной установленной ценой у каждого хозяина. Укажите топ-10 хостов с наибольшей разницей. 

```sql
SELECT host_id,
       MAX(toFloat32OrNull(replaceRegexpAll(price, '[$,]', ''))) 
       - MIN(toFloat32OrNull(replaceRegexpAll(price, '[$,]', ''))) AS diff_price
FROM listings
GROUP BY host_id
ORDER BY diff_price DESC
LIMIT 10
```

### 5. Выведите средние значения цены за ночь, размера депозита и цены уборки по типу жилья.
- room_type – тип сдаваемого жилья 
- price – цена за ночь
- security_deposit – залог за сохранность имущества
- cleaning_fee – плата за уборку

```sql
SELECT room_type,
       AVG(toFloat32OrNull(replaceRegexpAll(price, '[$,]', ''))) AS avg_price,
       AVG(toFloat32OrNull(replaceRegexpAll(security_deposit, '[$,]', ''))) AS avg_deposit,
       AVG(toFloat32OrNull(replaceAll(cleaning_fee, '$', ''))) AS avg_cleaning_fee
FROM listings
GROUP BY room_type
ORDER BY avg_deposit DESC
```

### 6. В каком районе города средняя стоимость за ночь на Airbnb является наименьшей?

- price – цена за ночь
- neighbourhood_cleansed – район/округ города

```sql
SELECT neighbourhood_cleansed,
       AVG(toFloat32OrNull(replaceRegexpAll(price, '[$,]', ''))) AS avg_price
FROM listings
GROUP BY neighbourhood_cleansed
ORDER BY avg_price 
LIMIT 1
```

### 7. В каких районах Берлина средняя площадь жилья, сдаваемого целиком, наибольшая? Определите топ-3 района.

- neighbourhood_cleansed – район
- square_feet – площадь
- room_type – тип (нужный – 'Entire home/apt')


```sql
SELECT neighbourhood_cleansed,
       AVG(toFloat32OrNull(square_feet)) AS avg_ssquare_feet
FROM listings
WHERE room_type = 'Entire home/apt'
GROUP BY neighbourhood_cleansed
ORDER BY avg_square DESC
LIMIT 3
```

### 8. Какая из представленных комнат расположена ближе всего к центру города?

- id – идентификатор жилья (объявления)
- room_type – тип жилья ('Private room')
- latitude – широта
- longitude – долгота
- 52.5200 с.ш., 13.4050 в.д – координаты центра Берлина

```sql
SELECT id,
       geoDistance(52.5200, 13.4050, toFloat64OrNull(latitude), toFloat64OrNull(longitude)) as distance
FROM listings
WHERE room_type = 'Private room'
ORDER BY distance 
LIMIT 1
```

### 9. Оставьте только те объявления, в которых оценка на основе отзывов выше среднего, а число отзывов в месяц составляет строго меньше трёх. Затем отсортируйте по убыванию две колонки: сначала по числу отзывов в месяц, потом по оценке.

- review_scores_rating – оценка на основе отзывов
- reviews_per_month – число отзывов в месяц

```sql
SELECT id, 
       reviews_per_month,
       toFloat32OrNull(review_scores_rating) AS rating
FROM listings
WHERE toFloat32OrNull(review_scores_rating) > (SELECT AVG(toFloat32OrNull(review_scores_rating))
                                               FROM listings) 
      AND reviews_per_month < 3
ORDER BY reviews_per_month DESC, rating DESC
```

### 10. Посчитайте среднее расстояние до центра города и выведите идентификаторы объявлений о сдаче отдельных комнат, для которых расстояние оказалось меньше среднего. Результат отсортируйте по убыванию, тем самым выбрав комнату, которая является наиболее удаленной от центра, но при этом расположена ближе, чем остальные комнаты в среднем.


```sql
WITH 
(SELECT AVG(geoDistance(13.4050, 52.5200, toFloat64OrNull(longitude), toFloat64OrNull(latitude)))
FROM listings 
WHERE room_type = 'Private room') as avg_distance

SELECT id,
       host_id,
       geoDistance(13.4050, 52.5200, toFloat64OrNull(longitude), toFloat64OrNull(latitude)) as dist_city
FROM listings
WHERE room_type = 'Private room' AND
      dist_city < avg_distance
ORDER BY dist_city DESC
```

### 11. Отберите объявления из таблицы listings, которые:

- находятся на расстоянии от центра меньше среднего 
- обойдутся дешевле 100$ в день (price с учетом cleaning_fee, который добавляется к общей сумме за неделю)
- имеют последние отзывы (last_review), начиная с 1 сентября 2018 года
- имеют WiFi в списке удобств (amenities)

Отсортируйте полученные значения по убыванию review_scores_rating. 

```sql
WITH 
(SELECT AVG(geoDistance(13.4050, 52.5200, toFloat64OrNull(longitude), toFloat64OrNull(latitude))) 
FROM listings) as avg_distance

SELECT id,
       host_id,
       geoDistance(13.4050, 52.5200, toFloat64OrNull(longitude), toFloat64OrNull(latitude)) as distance,
       price,
       cleaning_fee,
       amenities
FROM listings
WHERE distance < avg_distance AND 
      toFloat32OrNull(replaceRegexpAll(price, '[$,]', '')) + 
          (toFloat32OrNull(replaceRegexpAll(cleaning_fee, '[$,]', ''))) / 7 < 100  AND
      toDateOrNull(last_review) >= '2018-09-01' AND
      multiSearchAnyCaseInsensitive(amenities, ['WiFi'])
ORDER BY toFloat32OrNull(review_scores_rating) DESC
```

### 12. Найдите в таблице ```calendar_summary``` те доступные (available='t') объявления, у которых число отзывов от уникальных пользователей в таблице ```reviews``` выше среднего.



Будем считать, что отзыв — это уникальный посетитель на уникальное жилье, не учитывая возможные повторные отзывы от того же посетителя.

```sql
WITH 
(SELECT AVG(unique_reviews) AS avg_revs
FROM
    (SELECT COUNT(DISTINCT reviews.reviewer_id) AS unique_reviews
    FROM reviews
    GROUP BY reviews.listing_id) 
) AS avg_reviews


SELECT listing_id,
       COUNT(DISTINCT reviews.id) AS unique_reviews
FROM calendar_summary JOIN reviews ON calendar_summary.listing_id = reviews.listing_id
WHERE available='t' 
GROUP BY calendar_summary.listing_id
HAVING unique_reviews > avg_reviews
ORDER BY listing_id
LIMIT 10
```

### 13. Создайте колонку с обозначением группы, в которую попадает жилье из таблицы `listings`:

- 'good', если в удобствах (`amenities`) присутствует кухня и система отмены (`cancellation_policy`) гибкая
- 'ok', если в удобствах есть кухня, но система отмены не гибкая
- 'not ok' во всех остальных случаях

Результат отсортируйте по новой колонке по возрастанию, установите ограничение в 5 строк.

```sql
SELECT  id,
        host_id,
        CASE
            WHEN multiSearchAnyCaseInsensitive(amenities, ['kitchen']) AND 
                 cancellation_policy = 'flexible' THEN 'good'
            WHEN multiSearchAnyCaseInsensitive(amenities, ['kitchen']) AND
                 cancellation_policy != 'flexible' THEN 'ok'
            ELSE 'not ok'
        END AS category
FROM listings
ORDER BY category 
LIMIT 5
```